<a href="https://www.inove.com.ar"><img src="https://raw.githubusercontent.com/InoveAlumnos/dataset_analytics_python/master/images/PA%20Banner.png" width="1000" align="center"></a>


# Ejercicio de clasificación con vecinos cercanos (KNN)

Ejemplo de clasificación utilizando vecinos cercanos para la clasificación de drogadas que debería tomar un pasiente según su historial clínico<br>

v2.0

### Objetivos: 
*   Preprocesar los datos (descarga, lectura, limplieza y filtrado).
*   Conocer como funciona el algoritmo clasificación con KNN (KNeighborsClassifier).
*   Evaluar el resultado el algoritmo clasificación con KNN (KNeighborsClassifier).

**KNN:** Clasificador que implementa el voto de k-vecinos más cercanos.

Fuente: https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html

In [ ]:
#Librerias a implementar
import os
import platform

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

# Recolectar datos
<img src="https://raw.githubusercontent.com/InoveAlumnos/dataset_analytics_python/master/images/Pipeline1.png" width="1000" align="middle">

### Código de descarga del dataset

In [ ]:
if os.access('Telecust1.csv', os.F_OK) is False:
    if platform.system() == 'Windows':
        !curl https://raw.githubusercontent.com/InoveAlumnos/dataset_analytics_python/master/Telecust1.csv > Telecust1.csv
    else:
        !wget Telecust1.csv https://raw.githubusercontent.com/InoveAlumnos/dataset_analytics_python/master/Telecust1.csv

### `Telecust1.csv.csv`:
El dataset **`Telecust1.csv.csv`** contiene diferentes tipos de clientes que consumen un servicio de telecomunicación, los cuales deseamos clasificar en 4 categorias.<br> [Dataset source](https://www.kaggle.com/prathamtripathi/customersegmentation)

- **region** --> region, ejemplo 2
- **tenure** --> grado permanencia, ejemplo 40
- **age** --> edad, ejemplo 52
- **income** --> ingresos o sueldo, ejemplo 50 (un número que no representa una moneda real
- **marital** --> si está casado o no
- **address** --> dirección
- **employ** --> empleo
- **retire** --> si está retirado o no
- **genero** --> 0 o 1 (no se sabe cual es cual)


# Procesar datos
<img src="https://raw.githubusercontent.com/InoveAlumnos/dataset_analytics_python/master/images/Pipeline2.png" width="1000" align="middle">

In [ ]:
# Una vez descargado el archivo en Colab.
# Leerlo con Pandas y el método read_csv
# Una vez extraida toda la información se almacena en df
# A partir de df y el método describe(), mostrará la descripción estadistica básica del archivo que se guardará en des
# Crear una fila nueva llamada Nan en el DataFrame  des,
# que indica la cantidad de datos tipo Nan que tiene cada columna.
# Para crear una nueva fila, se utilizará el operador loc, donde se indica el nombre
# de la nueva fila y con que valores se completará.
# La información será de los datos faltantes df.isna().sum()
# Crear una fila nueva llamada %Nan en el DataFrame des,
# Esta fila se completará con los porcentajes de Nan encontrados en cada columna.

df = pd.read_csv("Telecust1.csv")
des = df.describe()
des.loc['Nan'] = df.isna().sum()
des.loc['%Nan'] = (df.isna().mean())*100
des

In [ ]:
# Muestra las 5 primeras filas del DataFrame df
df.head()

In [ ]:
# Cantidad de filas y columnas con shape
# En la ubicación 0 corresponde a las filas
print('Cantidad de datos en observacion:', df.shape[0])

# Explorar datos
<img src="https://raw.githubusercontent.com/InoveAlumnos/dataset_analytics_python/master/images/Pipeline3.png" width="1000" align="middle">

In [ ]:
# Descripción estadistica básica del DataFrame df
df.describe()

In [ ]:
# Se accede a la columna "custcat" para contar la frecuencia de los valores únicos (Cuenta la cantidad de clientes que tiene cada servicio).
# como está repartida las categorias entre los clientes actuales
df['custcat'].value_counts()

In [ ]:
# Se representa graficamente la frecuencia de los servicios ofrecidos.
# Esto permite explorar que tan balanceado está el dataset.
# sns, alias de Seaborn
# countplot(), gráfico de barras
# Necesita toda la data
# Se especifica la columna a representar, en este caso "custcat"
sns.countplot(data=df, x="custcat")

# Muestra el gráfico
plt.show()

Se puede ver que el dataset está bastante balanceado, no habrá una tendencia marcada

In [ ]:
# Nos quedamos con aquellas columnas que podemos entender su relacion con el objetivo:'tenure', 'age', 'income', 'marital', 'retire', 'gender', 'custcat'
# Para acceder a las columnas mencionadas se accede al DataFrame df[] y como 
# son varias columnas se indican los nombres en una lista
# Almacenandose en el DataFrame df_clean
df_clean = df[['tenure', 'age', 'income', 'marital', 'retire', 'gender', 'custcat']]
df_clean

#### Normalización de los datos

Analizar cual es la distribución de los datos numéricos
- tenure
- age
- income

In [ ]:
# Se representa graficamente la distribución de las edades de los clientes
# Esto permite explorar que tan balanceado está el dataset.
# sns, alias de Seaborn
# displot(), gráfico de distribución
# Necesita toda la data
# Se especifica la columna a representar, en este caso "age"
sns.displot(data=df_clean, x='age')
# Muestra el gráfico
plt.show()

In [ ]:
# Permite explorar que tan balanceado está el dataset.
# sns, alias de Seaborn
# displot(), gráfico de distribución
# Necesita toda la data
# Se especifica la columna a representar, en este caso "tenure"
sns.displot(data=df_clean, x='tenure')
# Muestra el gráfico
plt.show()

In [ ]:
# Permite explorar que tan balanceado está el dataset.
# sns, alias de Seaborn
# displot(), gráfico de distribución
# Necesita toda la data
# Se especifica la columna a representar, en este caso "income"
sns.displot(data=df_clean, x='income')

# Muestra el gráfico
plt.show()

El "ingreso" sigue una distribución normal, pero con muchos outliers. Es por eso, que no utilizaremos el MinMaxScaler sino que se utilizará el StandardScaler a pesar de que "tenure" no siga una distribución normal

In [ ]:
# Normalización de datos
# Se importa la herramienta de sklearn.preprocessing como StandardScaler
from sklearn.preprocessing import StandardScaler

# Se crea una copia del DataFrame df_clean a df_norm
df_norm = df_clean.copy()

# Se crean los objetos; age_scaler, tenure_scaler y income_scaler a partir de la clase StandardScaler()
age_scaler = StandardScaler()
tenure_scaler = StandardScaler()
income_scaler = StandardScaler()

# Del DataFrame normalizado df_norm se emplea el método .loc para editar los datos de las columnas: age, tenure e income 
# Cada columna se completará con los datos normalizados
# Para ello, se utiliza cada objeto creado y accede al método .fit_transform()
# se indica la columna del DataFrame a normalizar 
# Al agregar .values, solo toma los valores, sin nombres de funciones (Los nombres de las columnas).
df_norm.loc[:, 'age'] = age_scaler.fit_transform(df[['age']].values)
df_norm.loc[:, 'tenure'] = tenure_scaler.fit_transform(df[['tenure']].values)
df_norm.loc[:, 'income'] = income_scaler.fit_transform(df[['income']].values)
df_norm.head()

# Entrenar modelo
<img src="https://raw.githubusercontent.com/InoveAlumnos/dataset_analytics_python/master/images/Pipeline4.png" width="1000" align="middle">

El primer paso es obtener los datos que serán la entrada del sistema (X) y los datos que serán la salida del modelo estimador (y)

In [ ]:
# Obtener los valores de X e y
# En X se almacenarán todos los valores de las columnas excepto los valores de la columna "custcat"
# axis=1 para que se elimine por filas
# En y sólo se almacena los valores de la columna "custcat", que será la columna objetivo.
# Para ello, se accede a la columna "custcat" del DataFrame df_norm usando corchetes.
# En ambos caso, se implementa el método values para obtener solo los valores y que no vengan incluidos los nombres de las columnas.
X = df_norm.drop('custcat', axis=1).values
y = df_norm['custcat'].values
X.shape

Siguiente paso es dividir el dataset en entrenamiento (train) y evaluación (test). Utilizaremos el criterio 70%30%

In [ ]:
# Se importa la herramienta de sklearn.model_selectionl como train_test_split
from sklearn.model_selection import train_test_split

# Fijamos un "random_state" constante para que siempre el dataset se parta de la misma forma
# para poder repetir los ensayos
# Ojo! Los dataset de train y test son array numpy
# Se importa la herramienta de la libreria  train_test_split()
# Necesita los valores de X e y
# test_size=0.3, permite indicar el porcentaje de valores para evaluar, equivalente a un 30%
# random_state=42,  es un número fijo que utilizan comunmente en documentación, significa que para cada ejecución del algoritmo 
#se genere nuevos valores aleatorios
# y los conjuntos de datos de entrenamiento y pruebas serán diferentes.

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

### Crear un modelo de clasificación con vecinos cercanos (KNN)



#### Búsqueda del número de vecinos cercanos

In [ ]:
# Se importa la herramienta de sklearn.neighbors como KNeighborsClassifier
# Se importa la métrica de sklearn.metrics como accuracy_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

K_MAX = 20
# Crea un array de 20 elementos donde todos son ceros
mean_acc = np.zeros((K_MAX))

# Bucle itera 20 veces
for i in range(K_MAX):
    # Entrenar el modelo en cada iteración
    # n_neighbors --> (k) número de vecinos cercanos
    clf = KNeighborsClassifier(n_neighbors=(i+1)).fit(X_train,y_train)

    # Prediccion
    y_hat = clf.predict(X_test)   

    # Evaluar el modelo
    # Lo guarda en mean_acc
    mean_acc[i] = accuracy_score(y_test, y_hat)

#Representación gráfica de la exactitud de todas las iteraciones
plt.plot(range(1, K_MAX+1), mean_acc,'darkBlue')
# Nombra a los ejes
plt.ylabel('Accuracy')
plt.xlabel('K')
plt.tight_layout()
# Muestra el gráfico
plt.show()

# Muestra el máximo accuracy que se almacenó en mean_acc
# Del máximo accuracy busca su ubicación con mean_acc.argmax()+1
print(f"La mejor exactitud se obtuvo con {mean_acc.max():.2f} con K={mean_acc.argmax()+1}")

In [ ]:
# Una vez obtenido la ubicación de máximo accurary, se utiliza como números
# de vecinos cercanos n_neighbors=13
# Se entrana el modelo clasificador KNN con el método .fit()
# Para luego utilizar .predict() a partir de clf
clf = KNeighborsClassifier(n_neighbors=13).fit(X_train,y_train)
y_hat = clf.predict(X_test)   

# Validar modelo
<img src="https://raw.githubusercontent.com/InoveAlumnos/dataset_analytics_python/master/images/Pipeline5.png" width="1000" align="middle">

In [ ]:
# Calcular la exactitud (accuracy)
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_hat, normalize=True)

In [ ]:
# Se utliza la matriz de confusión para evaluar la precisión de una clasificación.
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

# Necesita dos variables que contengan los valores a comparar
cm = confusion_matrix(y_test, y_hat)

# Código para realizar la representación gráfica con los resultados
# Se crea la varible cmd, que almacena visualization de la Confusion Matrix 
# Necesita la variable cm que contiene los resultados de la comparación entre los valores reales y predicción
# display_labels, se especifica las etiquetas de las categorias que se evalúan.
cmd = ConfusionMatrixDisplay(cm, display_labels=clf.classes_)


# Con cmd.plot se especifica el mapa de colores reconocido por matplotlib.
cmd.plot(cmap=plt.cm.Greens)

# Para mostrar la figura
plt.show()

# Utilizar modelo
<img src="https://raw.githubusercontent.com/InoveAlumnos/dataset_analytics_python/master/images/Pipeline6.png" width="1000" align="middle">

In [ ]:
# Supongamos que deseamos ver a que categoría pertenecemos
# dado los siguientes datos
age = 25
tenure = 4
income = df['income'].mean() # ganamos el promedio
marital = 0 # no estamos casados
retire = 0 # no estamos retirados
gender = 1 # solo algun genero

In [ ]:
# El scaler espera como entrada una matriz (filas y columnas)
# Por eso el doble corchete
age_numpy = np.array([[age]])

# Utilizamos float para convertir la matriz que retorna el scaler
# a un número
age_norm = float(age_scaler.transform(age_numpy))
tenure_norm = float(tenure_scaler.transform(np.array([[tenure]])))
income_norm = float(income_scaler.transform(np.array([[income]])))
# El sistema espera como entrada "X" en este caso una sola fila pero varias
# columnas, por eso el reshape(1, -1) donde el "-1" significa "varias"
# (el sistema determina cuantas)
X_prueba = np.array([tenure_norm, age_norm, income_norm, marital, retire, gender]).reshape(1, -1)
print('Shape:', X_prueba.shape)
print('Valores:\n', X_prueba)

In [ ]:
# Se hace la predicción, a partir del objeto clasificador creado "clf"
# Y con el método .predict()
mi_categoria = clf.predict(X_prueba)
mi_categoria

# Conclusión
<img src="https://raw.githubusercontent.com/InoveAlumnos/dataset_analytics_python/master/images/Pipeline7.png" width="1000" align="middle">

En este ejemplo se obtuvo muy poca performance, pero se pudo ver como comparar muchos modelos KNN con distintos "K" y a su vez como ingresar un dato nuevo para predecir una categoría. Se podría probar con otros clasificadores pero el problema radica en la falta de datos